In [0]:
import tensorflow as tf
import numpy as np

tf.enable_eager_execution()
# sess = tf.InteractiveSession()

In [9]:
data = np.random.normal(0, 1, (10000, 4))

def online_statistics(data):
    n = 0
    mean = 0
    M2 = 0

    for x in data:
        n = n + 1
        delta = x - mean
        mean = mean + delta/n
        M2 = M2 + delta*(x - mean)

    variance = M2/(n - 1)
    return mean, variance

online_mean, online_var = online_statistics(data)
print("Online mean: {0}, online std: {1}".format(online_mean, online_var))

offline_mean = data.mean(axis=0)
offline_var = data.var(axis=0)
print("Offline mean: {0}, offline std: {1}".format(offline_mean, offline_var))


Online mean: [-0.00243011 -0.00168271  0.00190514 -0.01624116], online std: [0.98260742 0.97492802 0.98959768 1.01408439]
Offline mean: [-0.00243011 -0.00168271  0.00190514 -0.01624116], offline std: [0.98250916 0.97483053 0.98949872 1.01398299]


In [2]:
seq_len = 10
feature_size = 3
batch_size = 4
num_epochs = 2
eval_frequency = 2  # in number of training steps.

# samples = np.random.randint(0, 10, (batch_size*5, seq_len, feature_size))
# labels = np.random.randint(0, 10, (batch_size*5, seq_len, feature_size))
# eval_samples = np.random.randint(10, 15, (batch_size*2, seq_len, feature_size))
# eval_labels = np.random.randint(10, 15, (batch_size*2, seq_len, feature_size))

# Non-sequential data.
samples = np.random.randint(0, 10, (batch_size*5, feature_size))
labels = np.random.randint(0, 10, (batch_size*5, feature_size))
eval_samples = np.random.randint(10, 15, (batch_size*2+1, feature_size))
eval_labels = np.random.randint(10, 15, (batch_size*2+1, feature_size))

# Listed numbers
samples = np.array(range(batch_size*5+3))
labels = np.array(range(batch_size*5+3))
eval_samples = np.array(range(batch_size*5)) + 100
eval_labels = np.array(range(batch_size*5)) + 100

def normalize(sample):
  sample["inputs"] = (sample["inputs"]-10)
  return sample

print(samples)

[ 0  1  2  3  4  5  6  7  8  9 10 11 12 13 14 15 16 17 18 19 20 21 22]


In [3]:
# Eager Mode v1: we don't need to create any iterators. 

training_dataset = tf.data.Dataset.from_tensor_slices({"inputs":samples, "targets":labels})
# training_dataset = training_dataset.batch(batch_size).repeat(num_epochs).shuffle(buffer_size=10)
training_dataset = training_dataset.map(normalize)
training_dataset = training_dataset.batch(batch_size)

eval_dataset = tf.data.Dataset.from_tensor_slices({"inputs":eval_samples, "targets":eval_labels})
eval_dataset = eval_dataset.batch(batch_size)

iterator = training_dataset.make_one_shot_iterator()
features = iterator.get_next()
print(features)
for batch in iterator:
    pass

print("End of the first epoch.")
iterator = training_dataset.make_one_shot_iterator()
features = iterator.get_next()
print(features)

"""
for epoch in range(num_epochs):
  print("Epoch " + str(epoch+1))
  for train_batch in training_dataset:
    print(train_batch["inputs"])

print("End of Training.")
"""

Instructions for updating:
Use `for ... in dataset:` to iterate over a dataset. If using `tf.estimator`, return the `Dataset` object directly from your input function. As a last resort, you can use `tf.compat.v1.data.make_one_shot_iterator(dataset)`.
Instructions for updating:
Colocations handled automatically by placer.
{'inputs': <tf.Tensor: id=24, shape=(4,), dtype=int64, numpy=array([-10,  -9,  -8,  -7])>, 'targets': <tf.Tensor: id=25, shape=(4,), dtype=int64, numpy=array([0, 1, 2, 3])>}
End of the first epoch.
{'inputs': <tf.Tensor: id=45, shape=(4,), dtype=int64, numpy=array([-10,  -9,  -8,  -7])>, 'targets': <tf.Tensor: id=46, shape=(4,), dtype=int64, numpy=array([0, 1, 2, 3])>}


'\nfor epoch in range(num_epochs):\n  print("Epoch " + str(epoch+1))\n  for train_batch in training_dataset:\n    print(train_batch["inputs"])\n\nprint("End of Training.")\n'

In [4]:
# Eager Mode v2: only make_one_shot_iterator is supported. Make one pass and 
# recreate the iterator. Not sure about the cost of creating an iterator.

training_dataset = tf.data.Dataset.from_tensor_slices({"inputs":samples, "targets":labels})
training_dataset = training_dataset.batch(batch_size)
training_dataset = training_dataset.shuffle(buffer_size=10)
training_iterator = training_dataset.make_one_shot_iterator()

eval_dataset = tf.data.Dataset.from_tensor_slices({"inputs":eval_samples, "targets":eval_labels})
eval_dataset = eval_dataset.batch(batch_size)
eval_iterator = eval_dataset.make_one_shot_iterator()

for epoch in range(num_epochs):
  # Training.
  try:
    for step in range(eval_frequency):
      tf_training_pl = training_iterator.get_next()
      train_batch = tf_training_pl["inputs"]
      print(train_batch)

    # Evaluation: make a full pass on the evaluation data.
    while True:
      try:
        tf_eval_pl = eval_iterator.get_next()
        eval_batch = tf_eval_pl["inputs"]
        print(eval_batch)
      except tf.errors.OutOfRangeError:
        eval_iterator = eval_dataset.make_one_shot_iterator()
        print("End of Evaluation.")
        break
  
  except tf.errors.OutOfRangeError:
    training_iterator = training_dataset.make_one_shot_iterator()
    print("Epoch " + str(epoch+1))
    
print("End of Training.")

tf.Tensor(
[[0 6 5]
 [2 1 9]
 [1 3 7]
 [3 1 2]], shape=(4, 3), dtype=int64)
tf.Tensor(
[[6 9 0]
 [0 4 1]
 [3 8 4]
 [8 7 9]], shape=(4, 3), dtype=int64)
tf.Tensor(
[[10 11 13]
 [11 10 11]
 [14 12 12]
 [12 11 13]], shape=(4, 3), dtype=int64)
tf.Tensor(
[[10 10 10]
 [10 11 10]
 [10 13 13]
 [10 14 11]], shape=(4, 3), dtype=int64)
tf.Tensor([[11 14 10]], shape=(1, 3), dtype=int64)
End of Evaluation.
tf.Tensor(
[[5 6 2]
 [2 0 1]
 [5 7 9]
 [0 7 0]], shape=(4, 3), dtype=int64)
tf.Tensor(
[[7 1 1]
 [0 8 6]
 [6 5 2]
 [3 6 0]], shape=(4, 3), dtype=int64)
tf.Tensor(
[[10 11 13]
 [11 10 11]
 [14 12 12]
 [12 11 13]], shape=(4, 3), dtype=int64)
tf.Tensor(
[[10 10 10]
 [10 11 10]
 [10 13 13]
 [10 14 11]], shape=(4, 3), dtype=int64)
tf.Tensor([[11 14 10]], shape=(1, 3), dtype=int64)
End of Evaluation.
End of Training.


End of Training.


In [0]:
# Static Mode: the evaluation iterator is reinitialized after a full pass.
# The training data is repeated for number of epochs and processed via a
# one shot iterator. We can also use an initializable iterator similar to 
# evaluation case.

training_dataset = tf.data.Dataset.from_tensor_slices({"inputs":samples, "targets":labels})
training_dataset = training_dataset.batch(batch_size)
training_dataset = training_dataset.repeat(num_epochs)
training_dataset = training_dataset.shuffle(buffer_size=10)
training_iterator = training_iterator.make_one_shot_iterator()
tf_training_pl = training_iterator.get_next()

eval_dataset = tf.data.Dataset.from_tensor_slices({"inputs":eval_samples, "targets":eval_labels})
eval_dataset = eval_dataset.batch(batch_size)
eval_iterator = eval_dataset.make_initializable_iterator()
tf_eval_pl = eval_iterator.get_next()


while True:
  # Training.
  try:
    for step in range(eval_frequency):
      train_batch = sess.run(tf_training_pl["inputs"])
      print(train_batch)

    # Evaluation: make a full pass on the evaluation data.
    sess.run(eval_iterator.initializer)
    while True:
      try:
        eval_batch = sess.run(tf_eval_pl["inputs"])
        print(eval_batch)
      except tf.errors.OutOfRangeError:
        print("End of Evaluation.")
        break
  
  except tf.errors.OutOfRangeError:
    print("End of Training.")
    break

RuntimeError: ignored

In [5]:
eval_iterator = eval_dataset.make_initializable_iterator()

Instructions for updating:
Use `for ... in dataset:` to iterate over a dataset. If using `tf.estimator`, return the `Dataset` object directly from your input function. As a last resort, you can use `tf.compat.v1.data.make_initializable_iterator(dataset)`.


RuntimeError: ignored